In [2]:
import os
from pathlib import Path

In [28]:
PROJECT_NAME = 'VA'

list_of_files = [
    f"{PROJECT_NAME}/__init__.py",
    f"{PROJECT_NAME}/components/__init__.py",
    f"{PROJECT_NAME}/components/data_ingestion.py",
    f"{PROJECT_NAME}/components/data_transformation.py",
    f"{PROJECT_NAME}/components/data_validation.py",
    f"{PROJECT_NAME}/components/model_trainer.py",
    f"{PROJECT_NAME}/components/model_evaluation.py",
    f"{PROJECT_NAME}/entity/__init__.py",
    f"{PROJECT_NAME}/config/__init__.py",
    f"{PROJECT_NAME}/config/configuration.py",
    f"{PROJECT_NAME}/utils/__init__.py",
    f"{PROJECT_NAME}/utils/utils.py",
    f"{PROJECT_NAME}/exception/__init__.py",
    f"{PROJECT_NAME}/logger/__init__.py",
    "app.py",
    "requirements.txt",
    "Dockerfile",
    ".dockerignore",
    "setup.py",
    "config/model.yaml",
    "config/schema.yaml"
]

In [29]:
for files in list_of_files:
    filepath = Path(files)
    filedir, filename = os.path.split(filepath)
    
    if filedir !="":
        os.makedirs(filedir, exist_ok=True)
    if (not os.path.exists(filepath)) or (os.path.getsize(filepath)==0):
        with open(filepath,'w') as f:
            pass
    else:
        print(f"File is already present at {filepath}")

In [48]:
import setuptools

with open("README.md","r",encoding = "utf-8") as f:
    DESC = f.read()

PCK_DESC = "Python packages for VA"
__version__="0.0.0"
SRC_REPO = "VA"
REPO_NAME = "MLOps-VA"
AUTHOR_EMAIL = "tysonbarretto1991@gmail.com"
AUTHOR_NAME = "tysonbarreto"


In [49]:
setuptools.setup(
    name=SRC_REPO,
    version=__version__,
    author=AUTHOR_NAME,
    author_email=AUTHOR_EMAIL,
    description=PCK_DESC,
    long_description=DESC,
    long_description_content_type="text/markdown",
    url=f"https://github.com/{AUTHOR_NAME}/{REPO_NAME}",
    project_urls={
        "Bugs Tracker": f"https://github.com/{AUTHOR_NAME}/{REPO_NAME}/issues"
    },
    package_dir = {"":"VA"},
    packages = setuptools.find_packages(where = "VA")
)

AttributeError: 'tuple' object has no attribute 'tb_frame'